In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from datetime import datetime
import time
import os
import warnings
from siml.detect_peaks import *
from scipy.signal import savgol_filter
from sklearn.metrics import mean_squared_error
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

input_folder = r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\Forecasting Product\DBM\inputs'
output_folder =  r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\Forecasting Product\DBM\outputs'

######################
# READ IN DATA
######################
input_file = '202211\confidence_metric.csv'
path = os.path.join(input_folder, input_file)
cm=pd.read_csv(path)

input_file = r'202211\units_act.csv'
path = os.path.join(input_folder, input_file)
units=pd.read_csv(path)

C:\Users\A4023862\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Find the tags of concern
cm1=cm[cm['Accuracy']<=50]
cm2=cm[cm['Accuracy']>50]
cm3=cm[~cm['Accuracy'].notnull()]

print(cm['Tag'].nunique())
print(cm1['Tag'].nunique())
print(cm2['Tag'].nunique())
print(cm3['Tag'].nunique())

687
201
292
194


In [3]:
periods = 24
max_date = datetime.strptime(max(units['ds']), '%Y-%m-%d')
min_date = max_date - relativedelta(months=periods)

max_date = max_date.strftime('%Y-%m-%d')
min_date = min_date.strftime('%Y-%m-%d')

units1 = units[(units['ds']>=min_date) & (units['ds']<=max_date)]

units1

,Unnamed: 0,tag,Cost Objects,Products,value,ds,no_sales,short_history,theta_check,data_periods
0,42477,JPY - Japan Yen_D_INTL_LEGACY_OTH - D_INTL_LEG...,D_INTL_LEGACY_OTH - D_INTL_LEGACY_OTH,P_ADG_TOT - Advagraf/GRA Total,0.0,2022-10-01,1,0,1,12
7,42453,JPY - Japan Yen_D_INTL_LEGACY_OTH - D_INTL_LEG...,D_INTL_LEGACY_OTH - D_INTL_LEGACY_OTH,P_ADG_TOT - Advagraf/GRA Total,0.0,2020-10-01,1,0,1,12
8,42454,JPY - Japan Yen_D_INTL_LEGACY_OTH - D_INTL_LEG...,D_INTL_LEGACY_OTH - D_INTL_LEGACY_OTH,P_ADG_TOT - Advagraf/GRA Total,0.0,2020-11-01,1,0,1,12
9,42455,JPY - Japan Yen_D_INTL_LEGACY_OTH - D_INTL_LEG...,D_INTL_LEGACY_OTH - D_INTL_LEGACY_OTH,P_ADG_TOT - Advagraf/GRA Total,0.0,2020-12-01,1,0,1,12
10,42456,JPY - Japan Yen_D_INTL_LEGACY_OTH - D_INTL_LEG...,D_INTL_LEGACY_OTH - D_INTL_LEGACY_OTH,P_ADG_TOT - Advagraf/GRA Total,0.0,2021-01-01,1,0,1,12
...,...,...,...,...,...,...,...,...,...,...
46827,45,LC - LC_D_US - US_P_XTD_TOT - Xtandi Total,D_US - US,P_XTD_TOT - Xtandi Total,18918.0,2021-01-01,0,0,1,67
46828,46,LC - LC_D_US - US_P_XTD_TOT - Xtandi Total,D_US - US,P_XTD_TOT - Xtandi Total,22168.0,2021-02-01,0,0,1,67
46829,47,LC - LC_D_US - US_P_XTD_TOT - Xtandi Total,D_US - US,P_XTD_TOT - Xtandi Total,31988.0,2021-03-01,0,0,1,67
46830,48,LC - LC_D_US - US_P_XTD_TOT - Xtandi Total,D_US - US,P_XTD_TOT - Xtandi Total,24313.0,2021-04-01,0,0,1,67


In [44]:
from tslearn.generators import random_walks
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans
import random

X = random_walks(n_ts=50, sz=32, d=1)
km = TimeSeriesKMeans(n_clusters=3, metric="euclidean", max_iter=5, random_state=0).fit(X)
km.cluster_centers_.shape

km_dba = TimeSeriesKMeans(n_clusters=3, metric="dtw", max_iter=5, max_iter_barycenter=5, random_state=0).fit(X)
km_dba.cluster_centers_.shape

km_sdtw = TimeSeriesKMeans(n_clusters=3, metric="softdtw", max_iter=5,max_iter_barycenter=5,metric_params={"gamma": .5},random_state=0).fit(X)
km_sdtw.cluster_centers_.shape

# X_bis = to_time_series_dataset([[1, 2, 3, 4],[1, 2, 3],[2, 5, 6, 7, 8, 9]])
# km = TimeSeriesKMeans(n_clusters=2, max_iter=5,metric="dtw", random_state=0).fit(X_bis)
# km.cluster_centers_.shape

(3, 32, 1)

In [102]:
# Get list of tags and manipulate time series into array of arrays
tags = units1['tag'].unique().tolist()
tags_x = random.sample(tags, int(len(tags)*.8))
tags_y = [i for i in tags if i not in tags_x]

# Filter dfs
units_x = units1[units1['tag'].isin(tags_x)]
units_y = units1[units1['tag'].isin(tags_y)]

# Create lists of the time series
ts_array_x = []
ts_array_y = []

for i in range(0,len(tags_x)):
    df = units_x[units_x['tag'] == tags_x[i]]['value']
    ts_array_x.append(df.to_numpy())
    
for i in range(0,len(tags_y)):
    df = units_y[units_y['tag'] == tags_y[i]]['value']
    ts_array_y.append(df.to_numpy())
    

# Convert all into arrays
ts_array_x = np.array(ts_array_x)
ts_array_y = np.array(ts_array_y)


# Cluster based on dynamic time warping
km_dba = TimeSeriesKMeans(n_clusters=3, metric="dtw", max_iter=5, max_iter_barycenter=5, random_state=0).fit(ts_array_x)


In [104]:
# Predict the cluster
q = km_dba.predict(ts_array_y)

data = pd.DataFrame({'tag': tags_y, 'cluster': q})
data = data.merge(units_y[['tag', 'Cost Objects', 'Products']].drop_duplicates().reset_index(drop=True), how='left', on='tag')

data

,tag,cluster,Cost Objects,Products
0,JPY - Japan Yen_D_INTL_LEGACY_OTH - D_INTL_LEG...,0,D_INTL_LEGACY_OTH - D_INTL_LEGACY_OTH,P_ADG_TOT - Advagraf/GRA Total
1,JPY - Japan Yen_D_INTL_LEGACY_OTH - D_INTL_LEG...,0,D_INTL_LEGACY_OTH - D_INTL_LEGACY_OTH,P_BENDAMUSTIN - Bendamustin
2,JPY - Japan Yen_D_INTL_LEGACY_OTH - D_INTL_LEG...,0,D_INTL_LEGACY_OTH - D_INTL_LEGACY_OTH,P_PRG_TOT - Prograf Total
3,JPY - Japan Yen_D_MBM - M&BM_P_XOS_TOT - Xospa...,0,D_MBM - M&BM,P_XOS_TOT - Xospata total
4,LC - LC_D_ACN - ACN (China)_P_ALK - Allelock,0,D_ACN - ACN (China),P_ALK - Allelock
...,...,...,...,...
135,LC - LC_D_TR - Turkey_P_HA_TOT - HA/Tocas/Omni...,0,D_TR - Turkey,P_HA_TOT - HA/Tocas/Omnic Total
136,LC - LC_D_US - US_P_AMBISOME - Ambisome,0,D_US - US,P_AMBISOME - Ambisome
137,LC - LC_D_US - US_P_LEXI - Lexiscan,0,D_US - US,P_LEXI - Lexiscan
138,LC - LC_D_US - US_P_PADCEV_TOT - PADCEV total,0,D_US - US,P_PADCEV_TOT - PADCEV total
